In [1]:
import ast
import os
from pprint import pprint

import numpy as np
import pandas as pd

from utils.helper import read_xml

In [2]:
data_path = os.path.join("..", "data")

In [3]:
cocktails = read_xml(os.path.join(data_path, "ccc_cocktails.xml"))
print(len(cocktails))
print(type(cocktails))

108
<class 'xml.etree.ElementTree.Element'>


In [4]:
df = pd.read_csv(
    os.path.join(data_path, "recipes_w_search_terms.csv"),
    encoding="utf-8",
    dtype={"id": int, "name": str, "description": str, "serving_size": str, "servings": np.uint8},
    converters={
        "ingredients": ast.literal_eval,
        "ingredients_raw_str": ast.literal_eval,
        "steps": ast.literal_eval,
        "tags": ast.literal_eval,
        "search_terms": ast.literal_eval,
    },
)

In [5]:
df.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","[water, grits, salt, cheddar cheese, garlic, o...","[4 cups water, 1 cup uncooked old fas...",1 (155 g),8,"[I a sauce pan, bring water to a boil; slowly ...","[time-to-make, course, main-ingredient, prepar...","{vegetarian, low-carb, diabetic, low-calorie, ..."
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","[onion, red bell pepper, garlic cloves, large ...","[1 medium onion, chopped coarse , 1 med...",1 (366 g),4,"[In a food processor, pulse the onion, red pep...","[60-minutes-or-less, time-to-make, course, mai...","{dinner, shrimp}"
2,41090,black-and-white bean salad,NaN,"[white beans, canned black beans, tomatoes, on...","[1 cup canned white beans, rinsed and drai...",1 (807 g),1,"[In a large bowl, combine beans, tomato, onion...","[15-minutes-or-less, time-to-make, course, mai...","{vegetarian, dinner, salad, vegan, side}"
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"[zucchini, yellow squash, diced tomatoes, onio...","[2 zucchini, sliced , 2 small yello...",1 (244 g),4,[Put all ingredients in the crock pot and cook...,"[weeknight, time-to-make, course, main-ingredi...","{side, italian, vegetarian}"
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"[beef stew meat, flour, salt, allspice, cinnam...","[3 lbs beef stew meat, 3 tablespoons ...",1 (358 g),8,"[Preheat oven to 350°F., Cut beef into 1 inch ...","[time-to-make, course, main-ingredient, prepar...",{dinner}


In [6]:
ingredient_lists = np.unique(df["ingredients"].to_numpy(copy=True))
ingredient_lists

array([list([]),
       list(['', '', 'blue curacao', 'cointreau liqueur', 'sugar', 'prosecco']),
       list(['', 'absolut vodka', 'kahlua', 'ice']), ...,
       list(['zwieback toast crumbs', 'butter', 'sugar', 'cream cheese', 'sugar', 'vanilla', 'lemon', 'lemon juice', 'eggs', 'sour cream', 'sugar', 'vanilla', 'lemon juice']),
       list(['zwieback toast crumbs', 'sugar', 'butter', 'cream cheese', 'half-and-half', 'canned pumpkin', 'sugar', 'all-purpose flour', 'vanilla', 'ground cinnamon', 'ground ginger', 'ground nutmeg', 'salt', 'eggs', 'sour cream', 'sugar', 'vanilla']),
       list(['zwieback toast crumbs', 'sugar', 'margarine', 'unflavored gelatin', 'water', 'lime juice', 'eggs', 'sugar', 'lime peel', 'lowfat neufchatel cheese', 'whipped topping'])],
      dtype=object)

In [9]:
ingredients = []
[ingredients.extend(sublist) for sublist in ingredient_lists]
ingredients = set(ingredients)
ingredients.remove("")
chars_to_replace = {
    "%e2%80%99": "'",
    "%e2%80%93": "-",
    "%c3%89": "é",
    "%c2%ae": "",
    "%26": "&",
    "%3b": ";",
    "&reg;": '',
    "%2f": "/",
    "%2c": ",",
    "%27": "'",
    "%25": "%",
    "%28": "(",
    "%29": ")",
}
for key, value in chars_to_replace.items():
    ingredients = [s.replace(key, value) for s in ingredients]

print(f"Number of ingredients:\n{len(ingredients)}")
print("Ingredients:")
pprint(ingredients)

Number of ingredients:
18619
Ingredients:
['dried mulberries',
 'dry soy candy sprinkle',
 'crystal light raspberry lemonade mix',
 'purple tomatoes',
 'bream',
 'caribou roast',
 'fresh thyme sprigs',
 'teddy grahams cinnamon graham snacks',
 'dole italian salad variety',
 'egg matzos',
 'locatelli cheese',
 'chili-garlic sauce',
 '18% table cream',
 'milk duds',
 '(5 oz.) fisher walnut halves and pieces',
 'kibbeh',
 'dried brown lentils',
 'dried shiitake mushrooms',
 'dill pickles',
 'candy oil',
 'cheese-filled spinach tortellini',
 'homemade type white bread',
 "campbell's turkey gravy",
 'potpourri',
 'dried hot red chili peppers',
 'reduced-fat strawberry yogurt',
 'chicken quarter',
 'green sweet peppers',
 'banana pepper juice',
 'natural-style chunky peanut butter',
 'vitamin',
 'honey dijon salad dressing',
 'martha white yellow cornbread mix',
 'sweet pepper',
 'green onion stems',
 'powdered oregano',
 'fat-free chocolate milk',
 'yellow food coloring',
 'whole wheat tort